In [1]:
import string
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import re


In [2]:
#proses read csv
data=pd.read_csv('/home/jack/Documents/111.csv')
data.head()

,ulasan,label
0,Kualitas produk luar biasa.,Positive
1,Barang terlambat sampai.,Negative
2,Barang tidak sesuai dengan deskripsi.,Negative
3,Harga yang sangat kompetitif.,Positive
4,Situs belanja yang mudah digunakan.,Positive


In [3]:
# Menghapus baris yang mengandung kolom kosong
data = data.dropna(subset=['ulasan'])

# Mengubah kolom 'ulasan' menjadi string jika belum berupa string
data['ulasan'] = data['ulasan'].apply(lambda x: str(x) if not isinstance(x, str) else x)

def casefolding(Ulasan):
    if not isinstance(Ulasan, str):
        Ulasan = str(Ulasan)
    Ulasan = Ulasan.lower()
    Ulasan = Ulasan.strip(" ")
    Ulasan = re.sub(r'[?|$|_|!2_:")(-+,]', '', Ulasan)
    Ulasan = re.sub(r'\d+', '', Ulasan)
    Ulasan = re.sub(r"\b[a-zA-Z]\b", "", Ulasan)
    Ulasan = re.sub('\s+', ' ', Ulasan)
    return Ulasan

# Menerapkan fungsi casefolding pada kolom 'ulasan'
data['ulasan'] = data['ulasan'].apply(casefolding)

# Menampilkan data teratas
data.head()

,ulasan,label
0,kualitas produk luar biasa.,Positive
1,barang terlambat sampai.,Negative
2,barang tidak sesuai dengan deskripsi.,Negative
3,harga yang sangat kompetitif.,Positive
4,situs belanja yang mudah digunakan.,Positive


In [4]:
def token(Ulasan):
    nstr = Ulasan.split(' ')
    nstr = [word for word in nstr if word != '']
    return nstr

data['ulasan'] = data['ulasan'].apply(token)
data.head()

,ulasan,label
0,"[kualitas, produk, luar, biasa.]",Positive
1,"[barang, terlambat, sampai.]",Negative
2,"[barang, tidak, sesuai, dengan, deskripsi.]",Negative
3,"[harga, yang, sangat, kompetitif.]",Positive
4,"[situs, belanja, yang, mudah, digunakan.]",Positive


In [5]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

def stopword_removal(Ulasan):
    filtering = stopwords.words('indonesian','english')
    x = []
    data = []
    def myFunc(x):
        if x in filtering:
            return False
        else:
            return True
    fit = filter(myFunc, Ulasan)
    for x in fit:
        data.append(x)
    return data
data['ulasan'] = data['ulasan'].apply(stopword_removal)
data.head()

[nltk_data] Downloading package stopwords to /home/jack/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,ulasan,label
0,"[kualitas, produk, biasa.]",Positive
1,"[barang, terlambat, sampai.]",Negative
2,"[barang, sesuai, deskripsi.]",Negative
3,"[harga, kompetitif.]",Positive
4,"[situs, belanja, mudah, digunakan.]",Positive


In [6]:
from sklearn.pipeline import Pipeline
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def stemming(Ulasan):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    do = []
    for w in Ulasan:
        dt = stemmer.stem(w)
        do.append(dt)
    d_clean=[]
    d_clean= " ".join(do)
    print (d_clean)
    return d_clean
data['ulasan'] = data['ulasan'].apply(stemming)

data.to_csv('data_clean.csv', index=False)
data_clean = pd.read_csv('data_clean.csv', encoding='latin1')
data_clean.head()

kualitas produk biasa
barang lambat sampai
barang sesuai deskripsi
harga kompetitif
situs belanja mudah guna
barang sesuai deskripsi
lambat rekomendasi
kualitas jelek kecewa
lambat rekomendasi
ulas produk sesuai nyata
barang sesuai deskripsi
kualitas produk biasa
puas beli ini
barang kirim cepat sesuai harap
kualitas produk biasa
barang lambat sampai
harga mahal kualitas
aplikasi belanja guna
lambat rekomendasi
lambat rekomendasi
beli lagi
harga kompetitif
barang kirim cepat sesuai harap
layan langgan ramah bantu
kualitas produk biasa
barang sesuai deskripsi
produk bagus puas
kualitas produk biasa
beli lagi
beli lagi
produk rusak terima
alam belanja senang
kualitas produk biasa
kecewa sesuai harap
layan langgan ramah bantu
layan langgan ramah bantu
rekomendasi teman puas
produk rusak terima
aplikasi belanja guna
produk bagus puas
ulas produk sesuai nyata
aplikasi belanja guna
layan langgan ramah bantu
harga kompetitif
aplikasi belanja guna
ulas produk sesuai nyata
puas beli ini
aplikas

,ulasan,label
0,kualitas produk biasa,Positive
1,barang lambat sampai,Negative
2,barang sesuai deskripsi,Negative
3,harga kompetitif,Positive
4,situs belanja mudah guna,Positive


In [7]:
data_clean = data_clean.astype({'label' : 'category'})
data_clean = data_clean.astype({'ulasan' : 'string'})
data_clean.dtypes

ulasan    string[python]
label           category
dtype: object

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer()
text_tf = tf.fit_transform(data_clean['ulasan'].astype('U'))
text_tf


<2000x39 sparse matrix of type '<class 'numpy.float64'>'
	with 6074 stored elements in Compressed Sparse Row format>

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(text_tf, data_clean['label'], test_size=0.2, random_state=42)

In [10]:
print("jumlah data uji : ",x_test.shape)
print("jumlah data latih : ",x_train.shape)

pos = (y_test == 'Positive').sum()
neg = (y_test == 'Negative').sum()
postrain = (y_train == 'Positive').sum()
negtrain = (y_train == 'Negative').sum()
total = pos + neg
print('jumlah data uji sentimen positif',pos)
print('jumlah data uji sentimen negatif',neg)
print('jumlah data latih sentimen positif',postrain)
print('jumlah data latih sentimen negatif',negtrain)

data_clean['label'].value_counts()


jumlah data uji :  (400, 39)
jumlah data latih :  (1600, 39)
jumlah data uji sentimen positif 197
jumlah data uji sentimen negatif 203
jumlah data latih sentimen positif 803
jumlah data latih sentimen negatif 797


label
Negative    1000
Positive    1000
Name: count, dtype: int64

In [11]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
tvec = CountVectorizer()
x_cvec = tvec.fit_transform(data_clean['ulasan'].values.astype('U'))
print(x_cvec)
h_tfidf = TfidfTransformer()
x_tfidf = h_tfidf.fit_transform(x_cvec)
print(x_tfidf)

x = data_clean.ulasan
y = data_clean.label

  (0, 19)	1
  (0, 27)	1
  (0, 7)	1
  (1, 4)	1
  (1, 21)	1
  (1, 32)	1
  (2, 4)	1
  (2, 34)	1
  (2, 10)	1
  (3, 13)	1
  (3, 18)	1
  (4, 35)	1
  (4, 5)	1
  (4, 25)	1
  (4, 11)	1
  (5, 4)	1
  (5, 34)	1
  (5, 10)	1
  (6, 21)	1
  (6, 30)	1
  (7, 19)	1
  (7, 15)	1
  (7, 16)	1
  (8, 21)	1
  (8, 30)	1
  :	:
  (1991, 18)	1
  (1992, 19)	1
  (1992, 13)	1
  (1992, 24)	1
  (1993, 5)	1
  (1993, 0)	1
  (1993, 33)	1
  (1994, 27)	1
  (1994, 31)	1
  (1994, 37)	1
  (1995, 30)	1
  (1995, 28)	1
  (1995, 36)	1
  (1996, 21)	1
  (1996, 30)	1
  (1997, 5)	1
  (1997, 0)	1
  (1997, 33)	1
  (1998, 19)	1
  (1998, 27)	1
  (1998, 7)	1
  (1999, 35)	1
  (1999, 5)	1
  (1999, 25)	1
  (1999, 11)	1
  (0, 27)	0.46696193288803906
  (0, 19)	0.5126843024195349
  (0, 7)	0.7204868904331719
  (1, 32)	0.6618418747843267
  (1, 21)	0.5609203917380088
  (1, 4)	0.49732649930849876
  (2, 34)	0.4697864843026071
  (2, 10)	0.7192425504973743
  (2, 4)	0.5118503811863659
  (3, 18)	0.7630706262299792
  (3, 13)	0.6463151084300812
  (4, 35)	0.

In [14]:
#####pastekan algoritma disini #####